# Data Transformation

This workbook is aimed at conducting an exploratory data analysis and cleaning process on a dataset of GPU models and their related features. The data is sourced from two distinct datasets: one detailing eBay listings of graphics cards, and another providing GPU benchmark statistics. The ultimate goal of the workbook is to understand the correlation between the performance of a GPU model (derived from its benchmark scores) and its corresponding price in the eBay market.

The first few steps in the workbook involve loading the datasets and performing an initial data examination. The author looks at various aspects of the datasets, such as the structure, the values within it, and the overall distribution of those values.

Next, the workbook focuses on data cleaning and preprocessing, removing any irrelevant or misleading records. This includes the removal of entries that are not GPU models and renaming model names for consistency and ease of understanding.

Subsequently, the author merges the two datasets based on the GPU models. This is done to ensure each model's pricing information and performance metrics are in the same record. Further cleaning and transformation steps are taken post-merging, such as dropping redundant columns and handling missing values.

Finally, the workbook introduces a new derived metric, 'performance_score,' calculated from the GPU's benchmark scores. This will be the primary performance metric in subsequent analyses.

### Loading the Data

In [19]:
import pandas as pd
import numpy as np

file_path = r"C:\Users\Lyagovich\Documents\Portfolio\Ebay Scraper\ebay_listings_cleaned.csv"
file_path2 = r"C:\Users\Lyagovich\Documents\Portfolio\Ebay Scraper\GPU_benchmarks_v7.csv"

df0 = pd.read_csv(file_path)
df = df0.copy()

df01 = pd.read_csv(file_path2)
df1 = df01.copy()

pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', None)
df.head()


,name,price,condition,brand,model,memory_GB,title,info,link
0,ASUS NVIDIA GeForce RTX 30...,259.99,Pre-Owned,ASUS,NVIDIA GeForce RTX 3060 Ti,8.0,New ListingASUS TUF NVIDIA...,Pre-Owned · ASUS · NVIDIA ...,https://www.ebay.com/itm/3...
1,NVIDIA NVIDIA GeForce RTX ...,232.50,Pre-Owned,NVIDIA,NVIDIA GeForce RTX 2080 Super,8.0,NVIDIA GeForce RTX 2080 Su...,Pre-Owned · NVIDIA · NVIDI...,https://www.ebay.com/itm/3...
2,XFX AMD Radeon RX 580 8 GB,79.95,Pre-Owned,XFX,AMD Radeon RX 580,8.0,XFX AMD Radeon RX 580 8GB ...,Pre-Owned · XFX · AMD Rade...,https://www.ebay.com/itm/2...
3,EVGA NVIDIA GeForce RTX 30...,225.00,Pre-Owned,EVGA,NVIDIA GeForce RTX 3060 Ti,8.0,EVGA GeForce RTX 3060 Ti X...,Pre-Owned · EVGA · NVIDIA ...,https://www.ebay.com/itm/2...
4,NVIDIA NVIDIA Quadro 4000 ...,180.00,Pre-Owned,NVIDIA,NVIDIA Quadro 4000,8.0,Nvidia Quadro P4000 8GB HP...,Pre-Owned · NVIDIA · NVIDI...,https://www.ebay.com/itm/4...


### Initial Data Examination

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4511 entries, 0 to 4510
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       4511 non-null   object 
 1   price      4511 non-null   float64
 2   condition  4511 non-null   object 
 3   brand      4511 non-null   object 
 4   model      4511 non-null   object 
 5   memory_GB  4511 non-null   float64
 6   title      4511 non-null   object 
 7   info       4511 non-null   object 
 8   link       4511 non-null   object 
dtypes: float64(2), object(7)
memory usage: 317.3+ KB


In [23]:
df['condition'].value_counts()

Pre-Owned                  3916
Open Box                    301
Brand New                   222
Refurbished                  71
Very Good - Refurbished       1
Name: condition, dtype: int64

In [24]:
df['model'].value_counts()

NVIDIA GeForce RTX 3080                     494
NVIDIA GeForce RTX 3070                     456
NVIDIA GeForce RTX 3060 Ti                  386
NVIDIA GeForce RTX 3060                     324
AMD Radeon RX 5700 XT                       225
NVIDIA GeForce RTX 3080 Ti                  218
NVIDIA GeForce GTX 1070                     215
NVIDIA GeForce RTX 3070 Ti                  198
AMD Radeon RX 580                           173
NVIDIA GeForce RTX 2060                     165
NVIDIA GeForce GTX 1080 Ti                  161
NVIDIA GeForce GTX 1060                     144
NVIDIA GeForce GTX 1660                     137
NVIDIA GeForce GTX 1080                     125
NVIDIA GeForce RTX 2070                     119
NVIDIA GeForce GTX 1050 Ti                  115
NVIDIA GeForce RTX 2080                      74
NVIDIA GeForce GTX 1070 Ti                   71
NVIDIA GeForce RTX 2080 Ti                   64
AMD Radeon RX 6800 XT                        60
NVIDIA Quadro 4000                      

### Data Cleaning

In [25]:
df = df[~df['model'].str.contains('GDDR6|chipset', case=False)]
df['model'].nunique()

72

In [26]:
df1['gpuName'].nunique()

2317

### Preparing the Benchmark Data

In [27]:
def add_brand(gpu_name):
    if gpu_name.startswith('GeForce'):
        return 'NVIDIA ' + gpu_name
    elif gpu_name.startswith('Radeon'):
        return 'AMD ' + gpu_name
    else:
        return gpu_name

df1['gpuName'] = df1['gpuName'].apply(add_brand)
df1 = df1[df1['gpuName'].str.startswith('NVIDIA') | df1['gpuName'].str.startswith('AMD')]

### Merging the Datasets

In [28]:
df_merged = pd.merge(df, df1, left_on='model', right_on='gpuName', how='left')
df_merged.head(500)

,name,price_x,condition,brand,model,memory_GB,title,info,link,gpuName,G3Dmark,G2Dmark,price_y,gpuValue,TDP,powerPerformance,testDate,category
0,ASUS NVIDIA GeForce RTX 30...,259.99,Pre-Owned,ASUS,NVIDIA GeForce RTX 3060 Ti,8.0,New ListingASUS TUF NVIDIA...,Pre-Owned · ASUS · NVIDIA ...,https://www.ebay.com/itm/3...,NVIDIA GeForce RTX 3060 Ti,20206.0,961.0,599.99,33.68,200.0,101.03,2020.0,Desktop
1,NVIDIA NVIDIA GeForce RTX ...,232.50,Pre-Owned,NVIDIA,NVIDIA GeForce RTX 2080 Super,8.0,NVIDIA GeForce RTX 2080 Su...,Pre-Owned · NVIDIA · NVIDI...,https://www.ebay.com/itm/3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XFX AMD Radeon RX 580 8 GB,79.95,Pre-Owned,XFX,AMD Radeon RX 580,8.0,XFX AMD Radeon RX 580 8GB ...,Pre-Owned · XFX · AMD Rade...,https://www.ebay.com/itm/2...,AMD Radeon RX 580,8907.0,772.0,349.00,25.52,185.0,48.14,2017.0,Desktop
3,EVGA NVIDIA GeForce RTX 30...,225.00,Pre-Owned,EVGA,NVIDIA GeForce RTX 3060 Ti,8.0,EVGA GeForce RTX 3060 Ti X...,Pre-Owned · EVGA · NVIDIA ...,https://www.ebay.com/itm/2...,NVIDIA GeForce RTX 3060 Ti,20206.0,961.0,599.99,33.68,200.0,101.03,2020.0,Desktop
4,NVIDIA NVIDIA Quadro 4000 ...,180.00,Pre-Owned,NVIDIA,NVIDIA Quadro 4000,8.0,Nvidia Quadro P4000 8GB HP...,Pre-Owned · NVIDIA · NVIDI...,https://www.ebay.com/itm/4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,EVGA NVIDIA GeForce RTX 30...,243.00,Pre-Owned,EVGA,NVIDIA GeForce RTX 3070,8.0,EVGA GeForce RTX 3070 XC3 ...,Pre-Owned · EVGA · NVIDIA ...,https://www.ebay.com/itm/1...,NVIDIA GeForce RTX 3070,22093.0,969.0,719.99,30.69,220.0,100.42,2020.0,Desktop
6,ZOTAC NVIDIA GeForce GTX 1...,70.00,Pre-Owned,ZOTAC,NVIDIA GeForce GTX 1060,6.0,New ListingZOTAC GeForce G...,Pre-Owned · ZOTAC · NVIDIA...,https://www.ebay.com/itm/3...,NVIDIA GeForce GTX 1060,10070.0,762.0,279.99,35.97,120.0,83.92,2016.0,Desktop
7,ASUS NVIDIA GeForce RTX 30...,279.99,Open Box,ASUS,NVIDIA GeForce RTX 3060,12.0,ASUS GeForce RTX 3060 12GB...,Open Box · ASUS · NVIDIA G...,https://www.ebay.com/itm/3...,NVIDIA GeForce RTX 3060,16958.0,951.0,329.00,51.55,170.0,99.76,2021.0,Desktop
8,ZOTAC NVIDIA GeForce RTX 2...,217.79,Pre-Owned,ZOTAC,NVIDIA GeForce RTX 2070,8.0,ZOTAC RTX 2070 8GB Blower ...,Pre-Owned · ZOTAC · NVIDIA...,https://www.ebay.com/itm/1...,NVIDIA GeForce RTX 2070,16079.0,833.0,627.44,25.63,175.0,91.88,2018.0,Desktop
9,EVGA NVIDIA GeForce GTX 10...,60.00,Pre-Owned,EVGA,NVIDIA GeForce GTX 1060,6.0,EVGA NVIDIA GeForce GTX 10...,Pre-Owned · EVGA · NVIDIA ...,https://www.ebay.com/itm/2...,NVIDIA GeForce GTX 1060,10070.0,762.0,279.99,35.97,120.0,83.92,2016.0,Desktop


### Data Cleaning and Transformation

In [29]:
df_merged = df_merged.dropna(subset=['gpuName'])
df_merged = df_merged.drop(['gpuName', 'link', 'testDate', 'TDP', 'title', 'info'], axis=1)
df_merged['model'] = df_merged['model'].apply(lambda x: ' '.join(x.split(' ')[1:]))
df_merged = df_merged.drop(['price_y'], axis=1)
df_merged = df_merged.rename(columns={'price_x': 'price'})

In [31]:
df_merged.head()

,name,price,condition,brand,model,memory_GB,G3Dmark,G2Dmark,gpuValue,powerPerformance,category
0,ASUS NVIDIA GeForce RTX 30...,259.99,Pre-Owned,ASUS,GeForce RTX 3060 Ti,8.0,20206.0,961.0,33.68,101.03,Desktop
2,XFX AMD Radeon RX 580 8 GB,79.95,Pre-Owned,XFX,Radeon RX 580,8.0,8907.0,772.0,25.52,48.14,Desktop
3,EVGA NVIDIA GeForce RTX 30...,225.00,Pre-Owned,EVGA,GeForce RTX 3060 Ti,8.0,20206.0,961.0,33.68,101.03,Desktop
5,EVGA NVIDIA GeForce RTX 30...,243.00,Pre-Owned,EVGA,GeForce RTX 3070,8.0,22093.0,969.0,30.69,100.42,Desktop
6,ZOTAC NVIDIA GeForce GTX 1...,70.00,Pre-Owned,ZOTAC,GeForce GTX 1060,6.0,10070.0,762.0,35.97,83.92,Desktop


### Creating Performance Score

In [32]:
df_merged['performance_score'] = (df_merged['G2Dmark'] * 10 + df_merged['G3Dmark']) / 2
df_merged.drop(['G2Dmark', 'G3Dmark', 'gpuValue'], axis=1, inplace=True)

In [34]:
df_merged.reset_index(drop=True, inplace=True)
df_merged.head()

,name,price,condition,brand,model,memory_GB,powerPerformance,category,performance_score
0,ASUS NVIDIA GeForce RTX 30...,259.99,Pre-Owned,ASUS,GeForce RTX 3060 Ti,8.0,101.03,Desktop,14908.0
1,XFX AMD Radeon RX 580 8 GB,79.95,Pre-Owned,XFX,Radeon RX 580,8.0,48.14,Desktop,8313.5
2,EVGA NVIDIA GeForce RTX 30...,225.00,Pre-Owned,EVGA,GeForce RTX 3060 Ti,8.0,101.03,Desktop,14908.0
3,EVGA NVIDIA GeForce RTX 30...,243.00,Pre-Owned,EVGA,GeForce RTX 3070,8.0,100.42,Desktop,15891.5
4,ZOTAC NVIDIA GeForce GTX 1...,70.00,Pre-Owned,ZOTAC,GeForce GTX 1060,6.0,83.92,Desktop,8845.0


### Exporting Cleaned Data

In [35]:
output_file_path = r"C:\Users\Lyagovich\Documents\Portfolio\GPU Project\Data.csv"
df_merged.to_csv(output_file_path, index = False)

# Conclusion

This workbook effectively demonstrates the process of cleaning, preprocessing, and merging two distinct datasets. The resultant merged dataset combines GPU market pricing and performance metrics, preparing the groundwork for future exploratory data analysis, price prediction modeling, or market segmentation tasks. The addition of the 'performance_score' metric provides a simplified representation of GPU performance, vital for understanding the performance-price relationship.
